In [18]:
import pandas as pd

JV = pd.read_json("JuzneVesti-SR.v1.0.jsonl", lines=True)
df = pd.read_json("023_JV_segments_matched_speaker_id_added.jsonl", lines=True)
JV.columns

Index(['path', 'orig_file', 'start', 'end', 'words', 'norm_words',
       'norm_words_start_times', 'utterance_id_start', 'utterance_id_end',
       'speaker_info', 'split'],
      dtype='object')

In [19]:
# DF: 
# key: segment_file
# values: speaker_breakdown
# JV:
# key: path
# values: speaker_info.speaker_breakdown


JV = JV.merge(
    right=df[["segment_file", "speaker_breakdown", "guest_name", "host", "guest_description"]],
    right_on="segment_file", 
    left_on="path"
        ).drop(
            columns=["segment_file", "utterance_id_start", "utterance_id_end"]
)
dicts = []
for i, row in JV.iterrows():
    curdict = {
        key: row[key] for key in ["speaker_breakdown",  "guest_name", "host", "guest_description"]
           }
    curdict["speaker_breakdown"] = row["speaker_breakdown"]
    dicts.append(curdict)
JV["speaker_info"] = dicts

JV = JV.drop(
    columns=["speaker_breakdown","guest_name", "host", "guest_description"]
)

In [20]:
speaker_infos = []
for i, row in JV.iterrows():
    curdict = row["speaker_info"]
    curbreakdown = curdict["speaker_breakdown"]
    try:
        curbreakdown = [i for i in curbreakdown if i[1] != i[2]]
    except Exception as e:
        print(e)
    finally:
        curdict = {**curdict, "speaker_breakdown": curbreakdown}
        speaker_infos.append(curdict)
JV["speaker_info"] = speaker_infos


In [21]:
curdict

{'speaker_breakdown': [['guest', 249.19, 252.89],
  ['host', 252.89, 261.42],
  ['guest', 261.42, 264.18]],
 'guest_name': 'Dušan Milisavljević',
 'host': 'Marija Vučić',
 'guest_description': 'narodni poslanik DS iz Niša'}

In [22]:
pd.set_option("display.max_colwidth", None)
JV.sample(6).T

,9203,5240,4622,6220,1241,5730
path,seg_audio/MbrIJyz1Oko__100.4__119.1__.wav,seg_audio/14AnXmxhHMc__701.8__717.5__.wav,seg_audio/PGxnCPcn5Ig__551.4__569.3__.wav,seg_audio/RIM74lYTyQM__258.0__274.7__.wav,seg_audio/XlkPlKXSHAM__374.0__391.1__.wav,seg_audio/ao2BlYF0sWo__296.4__313.0__.wav
orig_file,audio/MbrIJyz1Oko.wav,audio/14AnXmxhHMc.wav,audio/PGxnCPcn5Ig.wav,audio/RIM74lYTyQM.wav,audio/XlkPlKXSHAM.wav,audio/ao2BlYF0sWo.wav
start,100.4,701.8,551.4,258.0,374.0,296.4
end,119.1,717.5,569.3,274.7,391.1,313.0
words,"[koji, je, usvojio, neko,, odnosno, međunaordna, zajednica, oličena, u, UN,, koji, je, na, snazi, sem, (Rezolucije), 1244,, koji, kaže, da, je, to, teritorija, Srbije!, I, znate,, vaša, je, njiva,, iako, je, neko, uzurpirao,, radi, na, njoj,, seje,, sve, dotle, dok, vi, ne, odete, u, katastar, i, prepišete, da, je, to, njegovo.]","[ću, lepo, potražiti, mišljenje, ljudi, koji, su, od, strane, Grada, plaćeni,, iz, Centra, za, socijalni, rad,, oni, će, se, sigurno, složiti!, Ne, vidim, razloga, zašto, mi, ne, bi, sa, nekom, sumom, za, sledeću, godinu, planirali?!, Uvek, i, mora, da, postoji, malo, otvorena, komunikacija,, da, oko, tog, svratišta, ne, radimo, preko, medija,, nego, da, sednemo, i, da, se, odgovorimo.]","[Znate, kako., Verovatno, postoje, ljudi, koji, jesu, za, to, mesto, ali, samo, angažovanje, na, tom, mestu, podrazumeva, i, upliv, u, politiku., Onaj, ko, neće, da, se, prlja,, onbaj, ko, nešto, zna, uglavnom, neće, da, se, prlja.]","[Pre, svega, koliko, se, papira, troši, na, dostavljanje, materijala, narodnim, poslanicima,, koliko, sam, shvatio, taj, materijala, se, istovremeno,, paralelno, dostavlja, i, u, elektronskom, obliku., Ukazivali, ste, i, na, čuveno, zvono, u]","[misli, da, je, shodno., Nikada, se, vlast, u, Srbiji, nije, menjala, osim, na, predsedničkim, izborima,, to, je, još, jedan, pokazatelj, koliko, je, to, važno., Prethodni, predsednik, je, svoju, političku, snagu, stečenom, pobedom, 2012., godine, odlučio, da, iskorisit, tako, što, je, promenio, vlast., To]","[Videćemo, da, problem, nije, samo, u, Nišu,, problem, je, i, u, prestonici,, koja, se, postavlja, kao, tutor, prema, nama., Ja, ne, znam, kako, Gradonačelnik, ali, i, gradska, vlast, dozvoljava]"
norm_words,"[koji, je, usvojio, neko, odnosno, međunaordna, zajednica, oličena, u, un, koji, je, na, snazi, sem, rezolucije, jedna, tisuća, dvijesto, četrdeset, četiri, koji, kaže, da, je, to, teritorija, srbije, i, znate, vaša, je, njiva, iako, je, neko, uzurpirao, radi, na, njoj, seje, sve, dotle, dok, vi, ne, odete, u, katastar, i, prepišete, da, je, to, njegovo]","[ja, ću, lepo, potražiti, mišljenje, ljudi, koji, su, od, strane, grada, plaćeni, iz, centra, za, socijalni, rad, oni, će, se, sigurno, složiti, ne, vidim, razloga, zašto, mi, ne, bi, sa, nekom, sumom, za, sledeću, godinu, planirali, uvek, i, mora, da, postoji, malo, otvorena, komunikacija, da, oko, tog, svratišta, ne, radimo, preko, medija, nego, da, sednemo, i, da, se, odgovorimo]","[znate, kako, verovatno, postoje, ljudi, koji, jesu, za, to, mesto, ali, samo, angažovanje, na, tom, mestu, podrazumeva, i, upliv, u, politiku, onaj, ko, neće, da, se, prlja, onbaj, ko, nešto, zna, uglavnom, neće, da, se, prlja]","[pre, svega, koliko, se, papira, troši, na, dostavljanje, materijala, narodnim, poslanicima, koliko, sam, shvatio, taj, materijala, se, istovremeno, paralelno, dostavlja, i, u, elektronskom, obliku, ukazivali, ste, i, na, čuveno, zvono]","[misli, da, je, shodno, nikada, se, vlast, u, srbiji, nije, menjala, osim, na, predsedničkim, izborima, to, je, još, jedan, pokazatelj, koliko, je, to, važno, prethodni, predsednik, je, svoju, političku, snagu, stečenom, pobedom, dvije, tisuće, dvanaest, godine, odlučio, da, iskorisit, tako, što, je, promenio, vlast]","[videćemo, da, problem, nije, samo, u, nišu, problem, je, i, u, prestonici, koja, se, postavlja, kao, tutor, prema, nama, ja, ne, znam, kako, gradonačelnik, ali, i, gradska, vlast, dozvoljava]"
norm_words_start_times,"

In [23]:
df.columns

Index(['file', 'start', 'end', 'asr_transcription', 'kaldi_transcript',
       'guest_name', 'guest_description', 'host', 'kaldi_words',
       'kaldi_word_starts', 'kaldi_word_ends', 'segment_file',
       'average_distance', 'similarity_ratio', 'speaker_breakdown'],
      dtype='object')

In [24]:
for i, row in JV.iterrows():
    curdict = row["speaker_info"]
    curbreakdown = curdict["speaker_breakdown"]
    try:
        starts = [j[1] for j in curbreakdown]
        ends = [j[2] for j in curbreakdown]
        if len(starts) != len(set(starts)):
            print(i)
    except:
        continue

9984


In [25]:
JV.loc[9984, "speaker_info"]["speaker_breakdown"] = [['host', 467.95, 468.19],
                                                    ['guest', 468.19, 484.7],
                                                    ]

In [26]:
JV.loc[9984, "speaker_info"]

{'speaker_breakdown': [['host', 467.95, 468.19], ['guest', 468.19, 484.7]],
 'guest_name': 'Igor Novaković',
 'host': 'Predrag Blagojević',
 'guest_description': 'direktor “Jugoistoka”'}

In [27]:
for i, row in JV.iterrows():
    curdict = row["speaker_info"]
    curbreakdown = curdict["speaker_breakdown"]
    try:
        starts = [j[1] for j in curbreakdown]
        ends = [j[2] for j in curbreakdown]
        if len(starts) != len(set(starts)):
            print(i)
    except:
        continue

Are there `JV:` prompts in `words`?

In [28]:
"JV:" in " ".join(JV.words.apply(" ".join).tolist())

True

In [29]:
def filter(l:list)-> list:
    return [w.replace("<anchor_start>", "").replace("<anchor_end>", "").replace("JV:", "") for w in l if w != ""]

JV["norm_words"] = JV.norm_words.apply(filter)
JV["words"] = JV.words.apply(filter)

In [30]:
"JV:" in " ".join(JV.words.apply(" ".join).tolist())

False

In [31]:
norm_start_times = []
speaker_infos = []
for i, row in JV.iterrows():
    curdict = row["speaker_info"]
    start = row["start"]
    curbreakdown = [[j[0], 
                     round(j[1] - start, 2), 
                     round(j[2] - start, 2)] 
                     for j in curdict["speaker_breakdown"]]
    norm_start_times.append([round(j - start, 2) for j in row["norm_words_start_times"]])
    speaker_infos.append({**curdict, "speaker_breakdown": curbreakdown})
    if not len(row["norm_words"]) == len(row["norm_words_start_times"]):
        print(i)
JV["norm_words_start_times"] = norm_start_times
JV["speaker_info"] = speaker_infos
    

Checking if the number of word start times match the number of words:


In [ ]:
for i, row in JV.iterrows():
    if not len(row["norm_words"]) == len(row["norm_words_start_times"]):
        print(i)

In [ ]:
JV.to_json("JuzneVesti-SR.v1.0.1.jsonl", lines=True, orient="records")
